In [11]:
import mysql.connector
import csv

database = mysql.connector.connect(
    user='root', password='jarvis@admin', host='192.168.1.6')
cursor = database.cursor()

In [12]:
cursor.execute('CREATE DATABASE IF NOT EXISTS VGG_Face2')
cursor.execute('USE VGG_Face2')

In [13]:
cursor.execute('CREATE TABLE IF NOT EXISTS VGG_Face2_TRAIN (id INT AUTO_INCREMENT PRIMARY KEY, PATH_NAME VARCHAR(255), X INT, Y INT, W INT, H INT)')

cursor.execute('CREATE TABLE IF NOT EXISTS VGG_Face2_TEST (id INT AUTO_INCREMENT PRIMARY KEY, PATH_NAME VARCHAR(255), X INT, Y INT, W INT, H INT)')

In [14]:
with open(r'Y:\DATASETS\negative-faces\VGG-Face2\meta\bb_landmark\loose_bb_test.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)
    count = 0
    for row in reader:
        PATH_NAME = row[0]
        X = row[1]
        Y = row[2]
        W = row[3]
        H = row[4]

        cursor.execute(
            'INSERT INTO VGG_Face2_TEST (PATH_NAME,X,Y,W,H) VALUES (%s,%s,%s,%s,%s)', (PATH_NAME, X, Y, W, H))
        
        # print real time progress
        print(f'{count} records inserted', end='\r')
        count += 1

print('Commiting changes...')
database.commit()


KeyboardInterrupt: 

In [ ]:

cursor.close()
database.close()
